### clean data from PFAF

In [578]:
import pandas as pd
import re
import numpy as np
from collections import Counter

In [118]:
pfaf = pd.read_csv('pfaf_data.csv')

In [3]:
pfaf.to_csv('pfaf_data_original.csv', index=False)

In [119]:
pfaf = pfaf.drop_duplicates()

In [89]:
pfaf.columns

Index(['Care', 'Common Name', 'Cultivation details', 'Edibility Rating',
       'Edible Uses', 'Family', 'Found In', 'Genus', 'Habitats',
       'Known Hazards', 'Medicinal Rating', 'Medicinal Uses', 'Other Names',
       'Other Uses', 'Physical Characteristics', 'Propagation', 'Range',
       'Species', 'Summary', 'Synonyms', 'USDA hardiness', 'Weed Potential',
       'Unnamed: 22'],
      dtype='object')

In [90]:
for c in pfaf.columns:
    print(c, pfaf[c].isnull().sum())

Care 0
Common Name 4009
Cultivation details 4007
Edibility Rating 4005
Edible Uses 4007
Family 4005
Found In 5432
Genus 0
Habitats 4112
Known Hazards 4005
Medicinal Rating 4005
Medicinal Uses 4005
Other Names 5435
Other Uses 4005
Physical Characteristics 4005
Propagation 4027
Range 4006
Species 0
Summary 5145
Synonyms 4823
USDA hardiness 4005
Weed Potential 5449
Unnamed: 22 5577


### edible uses

In [91]:
pfaf['Edibility Rating'].unique()

array([nan, '   (1 of 5)', '   (3 of 5)', '   (0 of 5)', '   (2 of 5)',
       '   (4 of 5)', '   (5 of 5)'], dtype=object)

In [120]:
pfaf.fillna('', inplace=True)
pfaf['Edibility Rating'].replace('', '0', inplace=True)

In [121]:
pfaf['Edibility Rating'] = pfaf['Edibility Rating'].str.extract(r'([0-9])').astype(int)

In [122]:
pfaf['Edible Uses'].iloc[1]

'Edible Parts: Inner bark\nEdible Uses: Gum  Tea\n\nYoung shoot tips are used as a substitute for tea[177, 183]. The pitch obtained from the bark can be hardened (probably by immersing it in cold water[K]) and used as a chewing gum[257]. Inner bark[257]. No further information is given, but inner bark can be dried, ground into a powder and then used with grain flours etc to make bread and other preparations[257].'

In [123]:
edible_cols = pfaf['Edible Uses'].str.split('\n', expand=True)
edible_cols[['Genus', 'Species']] = pfaf[['Genus', 'Species']]
edible_cols.fillna('', inplace=True)

In [235]:
edible_parts_counts = Counter()
edible_uses_counts = Counter()
edible_cols[edible_cols[0].str.contains('Edible Parts')][0].str.split().apply(
    edible_parts_counts.update)
edible_cols[edible_cols[1].str.contains('Edible Uses')][1].str.split().apply(
    edible_uses_counts.update)
edible_parts_counts.most_common()

[('Edible', 1188),
 ('Parts:', 1188),
 ('Leaves', 400),
 ('Seed', 361),
 ('Fruit', 331),
 ('Root', 282),
 ('Flowers', 155),
 ('Stem', 86),
 ('Oil', 80),
 ('Inner', 59),
 ('bark', 59),
 ('Seedpod', 42),
 ('Sap', 35),
 ('Nectar', 5),
 ('Manna', 3),
 ('Pollen', 3),
 ('Apical', 1),
 ('bud', 1),
 ('Shoots', 1)]

In [134]:
edible_cols[edible_cols[0].str.contains('Nectar')]

,0,1,2,3,Genus,Species
106,Edible Parts: Nectar Seed,Edible Uses:,,Seed - cooked. Said to be as sweet as a chestn...,Aesculus,flava
1658,Edible Parts: Fruit Nectar,Edible Uses:,,"Fruit - raw[60, 105]. A sweet flavour[61]. The...",Comandra,umbellata
2858,Edible Parts: Nectar,Edible Uses: Tea,,The plant has been boiled up as a tea[257]. Th...,Ipomopsis,aggregata
3132,Edible Parts: Fruit Nectar,Edible Uses:,,"Fruit - raw or cooked[105, 177, 212]. Not tast...",Lonicera,ciliosa
3326,Edible Parts: Fruit Nectar,Edible Uses:,,"Fruit - fresh or dried[61, 105, 183]. The frui...",Menziesia,ferruginea


In [135]:
edible_cols[3].loc[2858]

'The plant has been boiled up as a tea[257]. The nectar is sucked from the flowers by children[257].'

In [136]:
edible_cols[edible_cols[0].str.contains('Manna')]

,0,1,2,3,Genus,Species
2987,Edible Parts: Manna Sap,Edible Uses: Gum Sweetener,,"A gum, or resin, is produced under the bark. I...",Larix,occidentalis
4303,Edible Parts: Inner bark Manna,Edible Uses: Coffee Condiment Gum Tea,,Young shoot tips - used as a flavouring in coo...,Pseudotsuga,menziesii
4653,Edible Parts: Flowers Inner bark Manna,Edible Uses: Tea,,A honeydew can be obtained from the cut branch...,Salix,gooddingii


In [137]:
edible_cols[edible_cols[0].str.contains('Pollen')]

,0,1,2,3,Genus,Species
4761,Edible Parts: Pollen Root Seed Stem,Edible Uses:,,Root - raw or cooked[172]. Rich in starch. Ste...,Scirpus,microcarpus
5378,Edible Parts: Flowers Leaves Oil Pollen Ro...,Edible Uses: Oil,,Roots - raw or cooked[145]. Rich in starch[105...,Typha,domingensis
5379,Edible Parts: Flowers Leaves Oil Pollen Ro...,Edible Uses: Oil,,"Roots - raw or cooked[2, 12]. They can be boil...",Typha,latifolia


In [138]:
edible_cols[(~edible_cols[0].str.contains('Edible Parts:')) & 
            ~(edible_cols[0]=='') & ~(edible_cols[0]=='None known') & 
            ~(edible_cols[0]=='0')][0].head()

76     Leaves - cooked[105]. Some caution is advised,...
101         Seeds[257]. No further information is given.
192    The heart of the plant is very rich in sacchar...
278        Catkins - raw or cooked. A bitter taste[172].
441    Young leaves - cooked[177]. Used as a potherb[...
Name: 0, dtype: object

In [139]:
edible_cols[3] = np.where(((~edible_cols[0].str.contains('Edible Parts:')) & 
            ~(edible_cols[0]=='') & ~(edible_cols[0]=='None known') & 
            ~(edible_cols[0]=='0')), edible_cols[0], edible_cols[3])

In [140]:
edible_cols.loc[76]

0          Leaves - cooked[105]. Some caution is advised,...
1                                                           
2                                                           
3          Leaves - cooked[105]. Some caution is advised,...
Genus                                                 Actaea
Species                                             racemosa
Name: 76, dtype: object

In [146]:
pfaf['Edible inner bark'] = edible_cols[0].str.lower().str.contains('inner bark')
pfaf['Edible roots'] = edible_cols[0].str.lower().str.contains('root')
pfaf['Edible seeds'] = edible_cols[0].str.lower().str.contains('seed')
pfaf['Edible seedpods'] = edible_cols[0].str.lower().str.contains('seedpod')
pfaf['Edible sap'] = (edible_cols[0].str.lower().str.contains('sap') | 
                      edible_cols[0].str.lower().str.contains('manna'))
pfaf['Edible leaves'] = edible_cols[0].str.lower().str.contains('leaves')
pfaf['Edible flowers'] = edible_cols[0].str.lower().str.contains('flowers')
pfaf['Edible fruit'] = edible_cols[0].str.lower().str.contains('fruit')
pfaf['Edible stems'] = edible_cols[0].str.lower().str.contains('stem')
pfaf['Edible oil'] = edible_cols[0].str.lower().str.contains('oil')
pfaf['Edible shoots'] = edible_cols[3].str.lower().str.contains('shoots')

In [236]:
edible_uses_counts.most_common()

[('Edible', 1188),
 ('Uses:', 1188),
 ('Tea', 174),
 ('Condiment', 120),
 ('Oil', 80),
 ('Gum', 61),
 ('Coffee', 58),
 ('Drink', 38),
 ('Sweetener', 36),
 ('Pectin', 7),
 ('Colouring', 7),
 ('Chocolate', 4),
 ('Curdling', 3),
 ('agent', 3),
 ('Milk', 2),
 ('Rutin', 1),
 ('Salt', 1),
 ('Stabilizer', 1),
 ('Gelatine', 1)]

In [149]:
pfaf['Edible use gum'] = edible_cols[1].str.lower().str.contains('gum')
pfaf['Edible use drink'] = (edible_cols[1].str.lower().str.contains('tea') | 
                            edible_cols[1].str.lower().str.contains('drink') |
                            edible_cols[1].str.lower().str.contains('coffee'))
pfaf['Edible use condiment'] = edible_cols[1].str.lower().str.contains('condiment')
pfaf['Edible use sweetener'] = edible_cols[1].str.lower().str.contains('sweetener')
pfaf['Edible use oil'] = edible_cols[1].str.lower().str.contains('oil')

In [164]:
pfaf['Edible description'] = edible_cols[3]

In [160]:
pfaf.drop('Edible Uses', axis=1, inplace=True)

In [150]:
pfaf.to_csv('pfaf_data.csv', index=False)

### medicinal uses

In [151]:
pfaf['Medicinal Rating'].unique()

array(['', '   (2 of 5)', '   (5 of 5)', '   (3 of 5)', '   (1 of 5)',
       '   (0 of 5)', '   (4 of 5)'], dtype=object)

In [153]:
pfaf['Medicinal Rating'].replace('', '0', inplace=True)
pfaf['Medicinal Rating'] = pfaf['Medicinal Rating'].str.extract(r'([0-9])').astype(int)

In [163]:
pfaf['Medicinal Uses'].iloc[1]

"Plants For A Future can not take any responsibility for any adverse effects from the use of plants. Always seek advice from a professional before using a plant medicinally.\n\n\nThis plant was used quite widely by native North American Indians. An infusion of the bark was used as a tonic and to treat stomach ailments, TB, haemorrhoids and various minor complaints[257]. The pitch, or resin, was also used to treat colds, sore throats etc[257]. The bark of this tree contains blisters that are filled with a resin called 'Canadian Balsam'[226]. Although the report does not mention the uses of this resin, it can almost certainly be used in the same ways as the resin of A. balsamea, as detailed below:- The resin obtained from this tree (see 'Uses notes' below) has been used throughout the world and is a very effective antiseptic and healing agent. It is used as a healing and analgesic protective covering for burns, bruises, wounds and sores[213, 222, 226]. It is also used to treat sore nippl

In [165]:
medicinal_cols = pfaf['Medicinal Uses'].str.split('\n', expand=True)
medicinal_cols[['Genus', 'Species']] = pfaf[['Genus', 'Species']]
medicinal_cols.fillna('', inplace=True)

In [166]:
medicinal_cols.head()

,0,1,2,3,4,5,Genus,Species
0,,,,,,,Abutilon,abutiloides
1,Plants For A Future can not take any responsib...,,,This plant was used quite widely by native Nor...,,,Abies,amabilis
2,Plants For A Future can not take any responsib...,Analgesic Antiscorbutic Antiseptic Diuretic...,,,The resin obtained from the balsam fir (see 'U...,,Abies,balsamea
6,,,,,,,Abutilon,berlandieri
7,,,,,,,Abies,bracteata


In [167]:
medicinal_cols[0].unique()

array(['',
       'Plants For A Future can not take any responsibility for any adverse effects from the use of plants. Always seek advice from a professional before using a plant medicinally.'],
      dtype=object)

In [237]:
medicinal_counter = Counter()
medicinal_cols[1].str.split().apply(medicinal_counter.update)
medicinal_cols[2].str.split().apply(medicinal_counter.update)
medicinal_counter.most_common()

[('Astringent', 248),
 ('Diuretic', 224),
 ('Poultice', 221),
 ('Tonic', 174),
 ('Febrifuge', 150),
 ('Stomachic', 132),
 ('Diaphoretic', 130),
 ('Ophthalmic', 95),
 ('Salve', 95),
 ('Antiseptic', 93),
 ('Laxative', 91),
 ('Emetic', 89),
 ('Analgesic', 86),
 ('Antirheumatic', 85),
 ('Expectorant', 81),
 ('Antispasmodic', 76),
 ('Skin', 72),
 ('Miscellany', 72),
 ('Pectoral', 71),
 ('Sedative', 66),
 ('Stimulant', 65),
 ('Odontalgic', 54),
 ('VD', 52),
 ('Carminative', 49),
 ('Cancer', 49),
 ('Anodyne', 48),
 ('Emmenagogue', 47),
 ('Birthing', 44),
 ('aid', 44),
 ('Vulnerary', 44),
 ('TB', 40),
 ('Cathartic', 40),
 ('Kidney', 40),
 ('Vermifuge', 38),
 ('Appetizer', 35),
 ('Alterative', 35),
 ('Anthelmintic', 34),
 ("Women's", 34),
 ('complaints', 34),
 ('Blood', 32),
 ('Parasiticide', 32),
 ('Hypotensive', 31),
 ('Antiinflammatory', 30),
 ('Cardiac', 30),
 ('Haemostatic', 29),
 ('Rubefacient', 28),
 ('Cardiotonic', 27),
 ('Purgative', 26),
 ('Contraceptive', 26),
 ('Stings', 26),
 ('Gal

In [197]:
to_remove = ['Alterative', 'Aromatherapy', 'Homeopathy', 'Antiscorbutic', 'Tonic', 
             'tonic', 'Blood', 'purifier', 'Cardiotonic', 'Cholagogue', 'Uterine', 
             'complaints', 'Antihalitosis', 'care', 'Miscellany', 'aid', 
             'Aromatic', 'Deobstruent', 'Acrid', 'Enuresis', 'Nutritive']

In [198]:
for use in medicinal_counter.keys():
    pfaf[use] = (medicinal_cols[1].str.contains(use) | 
                 medicinal_cols[2].str.contains(use))

In [199]:
pfaf['Laxative'] = (pfaf['Laxative'] | pfaf['Aperient'] | pfaf['Cathartic'])
to_remove.append('Aperient')
to_remove.append('Cathartic')
pfaf['Purgative'] = (pfaf['Purgative'] | pfaf['Hydrogogue'])
to_remove.append('Hydrogogue')
pfaf['Cancer'] = (pfaf['Cancer'] | pfaf['Antitumor'] | pfaf['Cytostatic'] | 
                  pfaf['Cytotoxic'] | pfaf['Resolvent'])
to_remove.append('Antitumor')
to_remove.append('Cytostatic')
to_remove.append('Cytotoxic')
to_remove.append('Resolvent')
pfaf['Fever'] = (pfaf['Antipyretic'] | pfaf['Febrifuge'])
to_remove.append('Antipyretic')
to_remove.append('Febrifuge')
pfaf['Stimulant'] = (pfaf['Stimulant'] | pfaf['Stimulants'])
to_remove.append('Stimulants')
pfaf['Birthing'] = (pfaf['Birthing'] | pfaf['Oxytoxic'])
to_remove.append('Oxytoxic')
pfaf['Female misc'] = pfaf["Women's"]
to_remove.append("Women's")

In [200]:
pfaf.drop(to_remove, axis=1, inplace=True)

In [183]:
pfaf['Medicinal description'] = (medicinal_cols[3] + 
    medicinal_cols[4] + medicinal_cols[5])

### habitats

In [234]:
habitats = pfaf['Habitats'].copy()
habitats_counts = Counter()
habitats.str.split(';').apply(habitats_counts.update)
habitats_counts.most_common()

[('', 5497),
 (' Cultivated Beds', 545),
 ('Woodland Garden Sunny Edge', 446),
 (' Dappled Shade', 335),
 (' Shady Edge', 272),
 ('Woodland Garden Canopy', 194),
 ('Woodland Garden Dappled Shade', 193),
 (' Bog Garden', 181),
 (' Ground Cover', 91),
 (' not Deep Shade', 82),
 (' Pond', 73),
 (' Sunny Edge', 62),
 ('Woodland Garden Secondary', 54),
 (' Hedge', 39),
 (' Secondary', 35),
 (' Meadow', 28),
 (' South Wall. By. West Wall. By.', 24),
 (' East Wall. By. South Wall. By.', 22),
 (' East Wall. By. South Wall. By. West Wall. By.', 11),
 (' Lawn', 8),
 (' Hedgerow', 8),
 (' East Wall. In. South Wall. In. West Wall. In.', 8),
 (' North Wall. In. East Wall. In. West Wall. In.', 4),
 (' North Wall. In. East Wall. In.', 4),
 (' South Wall. By.', 2),
 (' North Wall. By. East Wall. By.', 2),
 ('Woodland Garden Cultivated Beds', 1),
 ('Woodland Garden Shady Edge', 1),
 (' East Wall. In. West Wall. In.', 1),
 (' East Wall. By. West Wall. By.', 1),
 (' East Wall. In.', 1)]

In [159]:
pfaf['Canopy'] = habitats.str.contains('Canopy')
pfaf['Understory'] = habitats.str.contains('Secondary')
pfaf['Cultivated Beds'] = habitats.str.contains('Cultivated Beds')
pfaf['Dappled Shade'] = habitats.str.contains('Dappled Shade')
pfaf['Sunny Edge'] = habitats.str.contains('Sunny Edge')
pfaf['Ground Cover'] = habitats.str.contains('Ground Cover')
pfaf['Meadow'] = habitats.str.contains('Meadow')
pfaf['Shade Edge'] = habitats.str.contains('Shady Edge')
pfaf['Bog'] = habitats.str.contains('Bog')
pfaf['Hedge'] = habitats.str.contains('Hedge')
pfaf['Pond'] = habitats.str.contains('Pond')

In [161]:
pfaf.drop('Habitats', axis=1, inplace=True)

In [201]:
pfaf.to_csv('pfaf_data.csv', index=False)

### other uses

In [202]:
pfaf['Other Uses'].head()

0                                                     
1    Gum  Wood\n\nThe boughs are fragrant and can b...
2    Adhesive  Fibre  Gum  Kindling  Microscope  Re...
6                                                     
7                                                     
Name: Other Uses, dtype: object

In [209]:
other_uses_cols = pfaf['Other Uses'].str.split('\n', expand=True)

In [210]:
other_uses_cols[:10]

,0,1,2
0,,None,None
1,Gum Wood,,The boughs are fragrant and can be hung in the...
2,Adhesive Fibre Gum Kindling Microscope Re...,,"The balsamic resin 'Balm of Gilead'[11, 46] or..."
6,,None,None
7,,None,None
8,Dye Wood,,A tan coloured dye can be obtained from the ba...
9,Adhesive Gum Microscope Repellent Resin S...,,"Wood - light, soft, coarse grained, not strong..."
10,None known,None,None
11,,None,None
12,Baby care Dye Gum Incense Repellent Roofi...,,The aromatic leaves are used as a moth repelle...


In [211]:
other_uses_cols[1].describe()

count     803
unique      1
top          
freq      803
Name: 1, dtype: object

In [227]:
other_uses_cols.fillna('', inplace=True)

In [219]:
other_uses_cols[1] = np.where(other_uses_cols[0].str.contains('\.'), other_uses_cols[0], '')
other_uses_cols[0] = np.where(other_uses_cols[1]=='', other_uses_cols[0], '')

In [228]:
other_uses_cols[1] = other_uses_cols[1] + other_uses_cols[2]

In [230]:
other_uses_cols.drop(2, axis=1, inplace=True)

In [238]:
other_uses_counter = Counter()
other_uses_cols[0].str.split(' ').apply(other_uses_counter.update)
common_uses = other_uses_counter.most_common()
common_uses

[('', 5628),
 ('None', 584),
 ('known', 584),
 ('Dye', 274),
 ('Wood', 266),
 ('Repellent', 107),
 ('Fibre', 85),
 ('Oil', 80),
 ('Hedge', 78),
 ('Soil', 77),
 ('Tannin', 77),
 ('Miscellany', 72),
 ('Basketry', 71),
 ('Fuel', 70),
 ('stabilization', 66),
 ('Gum', 61),
 ('Soap', 50),
 ('Hair', 41),
 ('Herbicide', 38),
 ('Shelterbelt', 35),
 ('Incense', 32),
 ('Parasiticide', 32),
 ('Essential', 31),
 ('Latex', 30),
 ('Stuffing', 29),
 ('Weaving', 26),
 ('Paper', 25),
 ('Resin', 23),
 ('String', 23),
 ('Adhesive', 22),
 ('Disinfectant', 22),
 ('Insecticide', 22),
 ('Pioneer', 19),
 ('Broom', 18),
 ('Tinder', 18),
 ('Mordant', 17),
 ('Rootstock', 16),
 ('Charcoal', 15),
 ('Brush', 15),
 ('Waterproofing', 13),
 ('Pollution', 13),
 ('Companion', 12),
 ('Cosmetic', 12),
 ('Musical', 12),
 ('Preservative', 11),
 ('Thatching', 11),
 ('Beads', 11),
 ('Baby', 10),
 ('care', 10),
 ('Fungicide', 10),
 ('Wick', 9),
 ('Insulation', 9),
 ('Pitch', 9),
 ('Containers', 8),
 ('Needles', 8),
 ('Teeth', 8

In [240]:
pfaf['Oil other uses'] = other_uses_cols[0].str.contains('Oil')

In [242]:
del other_uses_counter['']
del other_uses_counter['None']
del other_uses_counter['known']
del other_uses_counter['Oil']
del other_uses_counter['Soil']
other_uses_counter['Soil stabilization'] = other_uses_counter['stabilization']
del other_uses_counter['stabilization']
other_uses_counter['Baby care'] = other_uses_counter['Baby']
del other_uses_counter['Baby']
del other_uses_counter['care']
other_uses_counter['Soil reclamation'] = other_uses_counter['reclamation']
del other_uses_counter['reclamation']
other_uses_counter['Liquid feed'] = other_uses_counter['Liquid']
del other_uses_counter['Liquid']
del other_uses_counter['feed']
other_uses_counter['Green manure'] = other_uses_counter['Green']
del other_uses_counter['Green']
del other_uses_counter['manure']
other_uses_counter['Rooting hormone'] = other_uses_counter['Rooting']
del other_uses_counter['Rooting']
del other_uses_counter['hormone']
del other_uses_counter['Deodorant']
other_uses_counter['Soil conditioner'] = other_uses_counter['conditioner']
del other_uses_counter['conditioner']
other_uses_counter['Friction sticks'] = other_uses_counter['Friction']
del other_uses_counter['Friction']
del other_uses_counter['sticks']
other_uses_counter['Fire retardant'] = other_uses_counter['Fire']
del other_uses_counter['Fire']
del other_uses_counter['retardant']
del other_uses_counter['paper']
other_uses_counter = {k:v for k,v in other_uses_counter.items() if k not in ['Besom',
                      'Darning', 'ball', 'Blotting', 'Other', 'Uses', 'Waxed', 'Fruit', 
                      'ripening', 'making']}

In [243]:
for k in other_uses_counter.keys():
    pfaf[k] = other_uses_cols[0].str.contains(k)

In [244]:
pfaf['Other uses description'] = other_uses_cols[1]
pfaf.drop('Other Uses', axis=1, inplace=True)

In [248]:
to_drop = []
pfaf['Fertilizer'] = (pfaf['Green manure'] | pfaf['Fertilizer'] | 
                      pfaf['Liquid feed'] | pfaf['Soil conditioner'])
to_drop = to_drop + ['Green manure', 'Liquid feed', 'Soil conditioner']
pfaf['Kindling'] = (pfaf['Kindling'] | pfaf['Tinder'])
to_drop.append('Tinder')
pfaf['Fibre'] = (pfaf['Fibre'] | pfaf['String'])
to_drop.append('String')
pfaf['Building material'] = (pfaf['Insulation'] | pfaf['Pipes'] | pfaf['Pitch'] | 
                             pfaf['Plaster'] | pfaf['Roofing'] | pfaf['Thatching'])
to_drop = to_drop + ['Insulation', 'Pipes', 'Pitch', 'Plaster', 'Roofing', 'Thatching']
pfaf['Sewing'] = (pfaf['Buttons'] | pfaf['Needles'] | pfaf['Pins'] | pfaf['Starch'] | 
                  pfaf['Stuffing'])
to_drop = to_drop + ['Buttons', 'Needles', 'Pins', 'Starch', 'Stuffing']
pfaf['Miscellany'] = (pfaf['Miscellany'] | pfaf['Microscope'] | pfaf['Bedding'] | 
                      pfaf['Brush'] | pfaf['Lining'] | pfaf['Packing'] | 
                      pfaf['Scourer'] | pfaf['Strewing'] | pfaf['Filter'] | 
                      pfaf['Straw'] | pfaf['Litmus'])
to_drop = to_drop + ['Microscope', 'Bedding', 'Brush', 'Lining', 'Packing', 'Scourer', 
                     'Strewing', 'Filter', 'Straw', 'Litmus']
pfaf['Weaving'] = (pfaf['Weaving'] | pfaf['Basketry'])
pfaf.rename({'Essential': 'Essential oil', 'Teeth': 'Tooth care', 
             'Shelterbelt': 'Windbreak', 'Rust': 'Rust treatment'})
pfaf.drop(to_drop, axis=1, inplace=True)

In [249]:
pfaf.to_csv('pfaf_data.csv', index=False)

### special uses

In [254]:
# special uses section was not captured by previous scrape. 
import selenium
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [272]:
options = Options()
options.headless = True
driver = Firefox(options=options)

In [283]:
special_uses_df = pd.DataFrame()
length = len(pfaf)
for i, (genus,species) in enumerate(pfaf[['Genus', 'Species']].values):
    if i % 100 == 0:
        driver.quit()
        driver = Firefox(options=options)
        print(f'\n {i}/{length}')
    elif i % 5 == 0:
        print('.', end='')
    special_uses = {'Genus': genus, 'Species': species}
    driver.get(f'https://pfaf.org/user/Plant.aspx?LatinName={genus}+{species}')
    special_uses['Special uses'] = driver.find_element_by_xpath('//h3[contains('
        'text(), "Special Uses")]/following-sibling::*/descendant::span').text
    special_uses_df = special_uses_df.append(special_uses, ignore_index=True)
driver.quit()
pfaf = pfaf.merge(special_uses_df, on=['Genus', 'Species'])

In [282]:
pfaf.to_csv('pfaf_data.csv', index=False)

In [307]:
special_use_counter = Counter()
pfaf['Special uses'].str.lower().str.split(' ').apply(special_use_counter.update)
special_use_counter.most_common()

[('', 5336),
 ('food', 335),
 ('forest', 335),
 ('scented', 201),
 ('plants', 201),
 ('ground', 112),
 ('cover', 112),
 ('attracts', 103),
 ('wildlife', 103),
 ('carbon', 90),
 ('farming', 90),
 ('hedge', 78),
 ('nitrogen', 74),
 ('fixer', 74),
 ('dynamic', 44),
 ('accumulator', 44)]

In [322]:
special_uses = ['Food forest', 'Scented plants', 'Ground cover', 'Attracts wildlife', 
                'Carbon farming', 'Nitrogen fixer', 'Dynamic accumulator']

In [323]:
for use in special_uses:
    pfaf[use] = pfaf['Special uses'].str.lower().str.contains(use.lower())

In [337]:
pfaf[special_uses].describe()

,Food forest,Scented plants,Ground cover,Attracts wildlife,Carbon farming,Nitrogen fixer,Dynamic accumulator
count,5577,5577,5577,5577,5577,5577,5577
unique,2,2,2,2,2,2,2
top,False,False,False,False,False,False,False
freq,5243,5376,5465,5474,5487,5503,5533


In [381]:
pfaf.drop('special uses', axis=1, inplace=True)

In [338]:
pfaf.to_csv('pfaf_data.csv', index=False)

### hardiness zone, care

In [352]:
hardiness = pfaf['USDA hardiness'].str.split(r'([0-9]+)-([0-9]+)',expand=True)
hardiness.head()

,0,1,2,3
0,,None,None,None
1,,4,8,
2,,3,6,
3,,None,None,None
4,,None,None,None


In [354]:
for i in hardiness.columns:
    print(hardiness[i].value_counts())

               5198
Coming soon     379
Name: 0, dtype: int64
4     394
3     230
5     201
6     127
7     110
8      60
2      32
10     19
9      16
1       4
Name: 1, dtype: int64
9     374
8     368
7     170
10    138
11     85
12     27
6      24
5       6
4       1
Name: 2, dtype: int64
    1193
Name: 3, dtype: int64


In [356]:
hardiness.drop([0, 3], axis=1, inplace=True)

In [359]:
pfaf['Min zone'] = hardiness[1]
pfaf['Max zone'] = hardiness[2]

In [366]:
pfaf = pfaf.rename({c: c.lower() for c in pfaf.columns}, axis=1)

In [379]:
pfaf.drop('usda hardiness', axis=1, inplace=True)

In [374]:
pfaf['care'][~(pfaf['care']=='')].head()

0                                                   []
1    ['Fully Hardy', 'Moist Soil', 'Full shade', 'S...
2    ['Fully Hardy', 'Moist Soil', 'Full shade', 'S...
3                                                   []
4                                                   []
Name: care, dtype: object

In [375]:
care_counter = Counter()

def get_care(l):
    care_counter.update(eval(l))
    
pfaf['care'].apply(get_care)
care_counter.most_common()

[('Moist Soil', 1494),
 ('Full sun', 1455),
 ('Fully Hardy', 1051),
 ('Semi-shade', 969),
 ('Well drained soil', 552),
 ('Wet Soil', 226),
 ('Full shade', 125),
 ('Frost Hardy', 116),
 ('Water Plants', 76),
 ('Half Hardy', 42),
 ('Tender', 25)]

In [376]:
for k in care_counter.keys():
    pfaf[k.lower()] = pfaf['care'].str.contains(k)

In [378]:
pfaf['moist soil'].describe()

count      5577
unique        2
top       False
freq       4083
Name: moist soil, dtype: object

In [380]:
pfaf.drop('care', axis=1, inplace=True)

In [382]:
pfaf.to_csv('pfaf_data.csv', index=False)

### cultivation details

In [562]:
import nltk
from nltk.corpus import stopwords, wordnet
from nltk import stem
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

In [465]:
pfaf = pd.read_csv('pfaf_data.csv')

In [400]:
def clean_strings(string):
    sentences = sent_tokenize(string)
    lemmas = []
    for i, sent in enumerate(sentences):
        sent = [word for word in re.findall('\w+', sent.lower()) 
                if word not in stopwords.words('english')]
        tagged = nltk.pos_tag(sent)
        lemm = stem.WordNetLemmatizer()
        sent = [lemm.lemmatize(t[0]) for t in tagged]
        lemmas.append(sent)
    return lemmas

In [501]:
cultivation = pd.DataFrame()
cultivation['text'] = pfaf['cultivation details'].replace({r'\[([0-9]+(, )*)+[0-9]*\]': 
                                                           ''}, regex=True)

In [503]:
cultivation['genus'] = pfaf['genus']
cultivation['species'] = pfaf['species']

In [502]:
cultivation.fillna('', inplace=True)

In [508]:
count_vec = CountVectorizer(stop_words='english')
count_vec.fit(cultivation['text'])

CountVectorizer(stop_words='english')

In [554]:
def vectorize(text, genus, species):
    sentences = sent_tokenize(text)
    vectors = count_vec.transform(sentences)
    vector_df = pd.DataFrame(data=vectors.toarray(), columns=count_vec.get_feature_names())
    vector_df['genus'] = genus
    vector_df['species'] = species
    return vector_df

In [558]:
all_sentences = pd.DataFrame()
for _, (text, genus, species) in enumerate(cultivation.values):
    if text:
        vector_df = vectorize(text, genus, species)
        all_sentences = all_sentences.append(vector_df, ignore_index=True)

In [559]:
all_sentences.shape

(14911, 4758)

In [563]:
names = all_sentences[['genus', 'species']]
all_sentences.drop(['genus', 'species'], axis=1, inplace=True)

In [564]:
km = KMeans()
clusters = km.fit_transform(all_sentences)

In [566]:
km.inertia_

92189.20476954358

In [567]:
scores = []
for i in range(5,40,5):
    km = KMeans(n_clusters=i)
    clusters = km.fit_transform(all_sentences)
    scores.append((i, km.inertia_, clusters))
    print(i, km.inertia_)

5 97778.18836337097
10 90594.74156603152
15 87027.0635657646
20 80407.23680482272
25 78519.69642141205
30 75761.57997907503
35 74275.23223286551


In [569]:
# looks like 20 is a good n

In [570]:
all_sentences['genus'] = names['genus']
all_sentences['species'] = names['species']

In [571]:
all_sentences.to_csv('all_cultivation_sentences.csv', index=False)

In [574]:
import pickle

In [573]:
with open('clusters.pkl', 'wb') as f:
    pickle.dump(scores, f)

### Analyze clusters

In [580]:
all_sentences = pd.read_csv('all_cultivation_sentences.csv')

In [575]:
with open('clusters.pkl', 'rb') as f:
    scores = pickle.load(f)
# oops forgot to include km in the pickle nvm

In [591]:
names = all_sentences[['genus', 'species']]
all_sentences.drop(['genus', 'species'], axis=1, inplace=True)

In [592]:
km = KMeans(n_clusters=20)
clusters = km.fit_predict(all_sentences)

In [594]:
all_sentences['cluster'] = clusters

In [ ]:
all_sentences.to_csv('all_cultivation_sentences.csv')